# 第10章 HMM
## By LiuGang - 2018/11/22
## Reference Book - statistical learning method (Chinese)

## No.1- problem for predicting<br>1:  Create some data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import copy
%matplotlib inline

# lamta = (A, B, Pai)
A = np.array([[0.5,0.2,0.3],
              [0.3,0.5,0.2],
              [0.2,0.3,0.5]])

B = np.array([[0.5,0.5],
              [0.4,0.6],
              [0.7,0.3]])

pai = np.array([0.2,0.4,0.4])

### 2: Class

In [417]:
class HMM():
    def __init__(self,):
        pass
    
    def __init_decode__(self, A, B, pai, O):
        self.A = A
        self.B = B
        self.pai = pai
        self.O = O
        self.T = len(self.O)
        self.N = len(self.A)
        self.delta = np.zeros((self.T,self.N))
        self.phi = np.zeros((self.T,self.N))
        
    
    def decode(self, A, B, pai, O):
        # step 1 initial
        self.__init_decode__(A, B, pai, O)
        self.delta[0] = self.pai * self.B[:,self.O[0]]
        self.phi[0] = self.phi[0] * 0
        # step 2 recurrence
        for t in range(1,self.T):
            for i in range(self.N):
                #print(np.max(self.delta[t-1] * self.A[:,i]), self.B[i,self.O[t]])
                self.delta[t][i] = np.max(self.delta[t-1] * self.A[:,i]) * self.B[i,self.O[t]]
                self.phi[t][i] = np.argmax(self.delta[t-1] * self.A[:,i])
        # step 3 stop
        Px = np.max(self.delta[self.T-1])
        ix = int(np.argmax(self.delta[self.T-1]))
        # step 4 backtracking
        Ix = [ix]
        for t in range(self.T-1)[::-1]:
            ix = int(self.phi[t+1][ix])
            Ix.append(ix)
        return Ix[::-1]
    
    def forward(self, A, B, pai, O):
        T = len(O)
        N = len(A)
        alpha = np.zeros((T, N))
        alpha[0] = pai[0] * B[:,O[0]]
        for t in range(T-1):
            for i in range(N):
                alpha[t+1][i] = np.dot(alpha[t], A[:,i]) * B[i][O[t+1]]
        return alpha.T
    
    def backward(self, A, B, pai, O):
        T = len(O)
        N = len(A)
        beta = np.zeros((T, N))
        beta[T-1] = 1
        for t in range(T-1)[::-1]:
            for i in range(N):
                beta[t][i] =np.dot(A[i] * B[:,O[t+1]] , np.array([beta[t+1]]).T)
        return beta.T
        
    def learning(self, O, A, B, pai, eposilon=0.05):
        """Baum-Weich算法"""
#         A = np.random.rand(2,2)
#         pai = np.random.rand(2)
#         B = np.random.rand(2,3)
        N = A.shape[0]
        T = len(O)
        
        while True:
            alpha = self.forward(A,B,pai,O)
            beta = self.backward(A,B,pai,O)
            
            kesi = np.zeros((N,N,T-1))
            for t in range(T-1):
                down = np.dot(np.dot(alpha[:,t].T, A) * B[:,O[t+1]].T, beta[:,t+1])
                for i in range(N):
                    up = alpha[i,t] * A[i,:] * B[:,O[t+1]].T * beta[:,t+1].T
                    kesi[i,:,t] = up / down
                    
            gamma = np.zeros((N, T))
            for t in range(T):
                gamma[:,t] = (alpha[:,t] * beta[:,t]) / np.dot(alpha[:,t],np.array([beta[:,t]]).T)

            newpai = gamma[:,0]
            newA = np.sum(kesi,2) / np.sum(gamma[:,:-1],axis=1).reshape((-1,1))
            newB = np.copy(B)
            num_levels = B.shape[1]
            sumgamma = np.sum(gamma,axis=1)
            for lev in range(num_levels):
                mask = O == lev
                newB[:,lev] = np.sum(gamma[:,mask],axis=1) / sumgamma

            # 阈值 stop
            if np.max(abs(pai - newpai)) < eposilon and \
                            np.max(abs(A - newA)) < eposilon and \
                            np.max(abs(B - newB)) < eposilon:
                break
            A, B, pai = newA, newB, newpai
        return newA, newB, newpai
            
            

### 3. Test

In [43]:
# I : [0,1,2]    ([1,2,3] in book)
# O : 0=red; 1=white
O = [0,1,0]
myhmm = HMM()
myhmm.decode(A,B,pai,O)

[2, 2, 2]

## No.2- problem for learning<br>1:  Create some data

In [418]:
import numpy as np
# 对应状态集合Q
states = ('Healthy', 'Fever')
# 对应观测集合V
observations = ('normal', 'cold', 'dizzy')
# 初始状态概率向量π
start_probability = {'Healthy': 0.6, 'Fever': 0.4}
pai = np.array([0.6, 0.4])
# 状态转移矩阵A
transition_probability = {
    'Healthy': {'Healthy': 0.7, 'Fever': 0.3},
    'Fever': {'Healthy': 0.4, 'Fever': 0.6},
}
A = np.array([[0.7,0.3],[0.4,0.6]])
# 观测概率矩阵B
emission_probability = {
    'Healthy': {'normal': 0.5, 'cold': 0.4, 'dizzy': 0.1},
    'Fever': {'normal': 0.1, 'cold': 0.3, 'dizzy': 0.6},
}
B = np.array([[0.5, 0.4, 0.1],[0.1, 0.3, 0.6]])
# 随机生成观测序列和状态序列    
def simulate(T):

    def draw_from(probs):
        return np.where(np.random.multinomial(1,probs) == 1)[0][0]

    observations = np.zeros(T, dtype=int)
    states = np.zeros(T, dtype=int)
    states[0] = draw_from(pi)
    observations[0] = draw_from(B[states[0],:])
    for t in range(1, T):
        states[t] = draw_from(A[states[t-1],:])
        observations[t] = draw_from(B[states[t],:])
    return observations, states

# 生成模拟数据
observations_data, states_data = simulate(600)
print('观测: ',observations_data)
print('状态: ',states_data)

观测:  [0 2 2 2 2 0 0 2 2 2 2 2 2 1 1 0 0 0 1 0 2 0 0 2 2 0 0 2 0 2 0 1 1 1 0 0 1
 1 2 2 2 1 2 2 2 1 1 1 2 0 2 0 1 1 0 2 0 0 2 2 0 2 1 2 2 0 1 0 1 2 0 0 1 0
 1 0 0 2 2 2 1 2 1 2 1 2 0 1 0 0 0 0 1 2 1 2 2 0 1 2 1 0 1 2 0 1 1 1 1 1 2
 2 2 1 0 0 0 1 1 1 2 1 0 2 2 2 1 0 1 0 0 0 1 1 0 0 0 1 2 1 2 2 1 1 0 0 1 0
 0 1 0 0 1 2 1 2 0 0 2 0 1 2 2 2 2 2 1 0 1 2 2 1 1 1 2 1 1 2 0 1 1 2 2 0 2
 2 0 0 1 0 1 2 1 2 2 2 2 0 0 2 1 0 0 2 2 2 0 2 0 0 1 1 1 2 2 0 0 2 0 0 0 0
 1 2 1 1 1 2 0 0 0 1 1 1 1 2 2 2 1 2 2 2 2 0 1 0 0 0 1 2 2 2 2 2 2 2 1 0 0
 0 0 2 2 2 1 1 0 2 0 1 1 1 1 0 2 0 1 0 2 1 2 2 2 0 1 0 1 1 1 1 0 0 1 1 2 0
 2 2 1 0 1 0 0 2 2 2 0 2 0 1 1 0 2 1 0 2 1 1 2 0 0 2 2 0 0 2 2 1 2 1 2 1 1
 2 2 1 2 0 1 2 2 0 1 0 0 1 1 1 0 2 1 2 2 1 1 0 0 1 0 0 0 2 1 0 1 1 0 1 1 0
 1 0 0 1 1 1 0 1 1 1 1 1 0 0 2 1 1 1 1 0 0 0 1 0 1 2 0 2 2 1 0 0 2 2 0 0 2
 1 1 0 2 1 2 2 1 1 2 2 1 0 0 1 2 0 0 0 1 0 0 0 0 1 1 1 2 2 0 1 1 1 2 0 0 0
 0 1 2 0 1 0 2 2 2 0 0 1 0 2 1 0 2 1 1 2 2 2 1 0 1 1 0 2 2 2 2 0 1 2 2 1 2
 0 1 0 0 1 1 0 0 0 1

In [420]:
myhmm = HMM()
# A = np.array([[0.1,0.1],[0.1,0.1]])
# pai = np.array([0.1, 0.1])
# B = np.array([[0.1, 0.1, 0.1],[0.1, 0.1, 0.1]])
#参数的初值影响很大，如果随机初值的话，结果偏差较大
newA, newB, newpai = myhmm.learning(observations_data, A, B, pai, eposilon=0.02)
print("newA: ", newA)
print("newB: ", newB)
print("newpi: ", newpai)

newA:  [[0.69440988 0.30559012]
 [0.35969474 0.64030526]]
newB:  [[0.50694641 0.38927474 0.10377885]
 [0.08830953 0.25999599 0.65169448]]
newpi:  [0.75895999 0.24104001]
